# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import os

# Import API key
#from api_keys import geoapify_key


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df["Cloudiness"] = (city_data_df["Cloudiness"]).str.replace("{'all':", "")
city_data_df["Cloudiness"] = (city_data_df["Cloudiness"]).str.replace("}", "")
city_data_df["City"] = city_data_df["City"].str.title()
city_data_df.head()

/var/folders/c6/f4lv068572x0t6lvnvn2yjyc0000gn/T/ipykernel_94587/318635296.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  city_data_df["Cloudiness"] = (city_data_df["Cloudiness"]).str.replace("{'all':", "")
/var/folders/c6/f4lv068572x0t6lvnvn2yjyc0000gn/T/ipykernel_94587/318635296.py:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  city_data_df["Cloudiness"] = (city_data_df["Cloudiness"]).str.replace("}", "")


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Atuona,-9.8000,-139.0333,27.21,76,59,6.58,PF,1680133060
1,1,Paciran,-6.8767,112.3761,26.58,83,100,3.55,ID,1680133062
2,2,Mahibadhoo,3.7833,72.9667,27.29,76,23,2.43,MV,1680133064
3,3,Sao Joao Da Barra,-21.6403,-41.0511,25.05,86,64,3.37,BR,1680133067
4,4,Talnakh,69.4865,88.3972,-27.61,100,100,2.92,RU,1680133070


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
#storing humidity as float tyope for the following plot
humidity = city_data_df["Humidity"].astype(float)

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    title = "City Humidity Heatmap"    
)

# Display the map plot
map_plot_humidity


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
#Store Cloudiness as int64
city_data_df["Cloudiness"] = city_data_df["Cloudiness"].astype("Int64")

# Narrow down cities that fit criteria and drop any results with null values
narrow_df = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 25) & (city_data_df["Cloudiness"] < 15) & (city_data_df["Wind Speed"] < 15)]

# Drop any rows with null values
narrow_df.dropna()

# Display sample data
narrow_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,29,Puri,19.8000,85.8500,26.29,85,7,7.10,IN,1680133013
47,47,Port Macquarie,-31.4333,152.9167,26.05,39,0,3.91,AU,1680133200
53,53,Arraial Do Cabo,-22.9661,-42.0278,25.92,83,0,5.14,BR,1680133209
86,86,Charters Towers,-20.1000,146.2667,28.34,56,6,4.43,AU,1680133260
95,95,Santa Cruz De Tenerife,28.4682,-16.2546,25.17,17,0,0.51,ES,1680133276


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


/var/folders/c6/f4lv068572x0t6lvnvn2yjyc0000gn/T/ipykernel_94587/572696565.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
0,Atuona,PF,-9.8000,-139.0333,76,
1,Paciran,ID,-6.8767,112.3761,83,
2,Mahibadhoo,MV,3.7833,72.9667,76,
3,Sao Joao Da Barra,BR,-21.6403,-41.0511,86,
4,Talnakh,RU,69.4865,88.3972,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
params = { 
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
 }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # Get the city's name
    city = row["City"]
    
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"
   
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

#Display sample data
hotel_df

Starting hotel search
atuona - nearest hotel: Pearl Resort
paciran - nearest hotel: No hotel found
mahibadhoo - nearest hotel: Coral View Inn
sao joao da barra - nearest hotel: Pousada Mediterrâneo
talnakh - nearest hotel: Талнах
port elizabeth - nearest hotel: Waterford Hotel
busselton - nearest hotel: Broadwater Beach Resort
half moon bay - nearest hotel: Half Moon Bay Inn
tiksi - nearest hotel: Арктика
werda - nearest hotel: No hotel found
sao paulo de olivenca - nearest hotel: No hotel found
avarua - nearest hotel: Paradise Inn
abha - nearest hotel: SAHAB
torbay - nearest hotel: Hampton
hithadhoo - nearest hotel: Pebbles Inn
port alfred - nearest hotel: No hotel found
barybino - nearest hotel: Мастер отель Домодедово
oxford - nearest hotel: Old Bank Hotel
xifeng - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
mahebourg - nearest hotel: Grand Bel Air
punta arenas - nearest hotel: Hotel Lacolet
rikitea - nearest hotel: Chez Bianca & Benoit
alingsas - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
0,atuona,PF,-9.8000,-139.0333,76,Pearl Resort
1,paciran,ID,-6.8767,112.3761,83,No hotel found
2,mahibadhoo,MV,3.7833,72.9667,76,Coral View Inn
3,sao joao da barra,BR,-21.6403,-41.0511,86,Pousada Mediterrâneo
4,talnakh,RU,69.4865,88.3972,100,Талнах
...,...,...,...,...,...,...
578,lata,PT,40.1629,-8.3327,94,Residencial Botânico
579,nauta,PE,-4.5061,-73.5757,69,No hotel found
580,tayginka,RU,55.6244,60.5076,77,Санаторий Дальняя дача
581,rapid valley,US,44.0625,-103.1463,57,Sleep Inn & Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    # by="Country",
    # hover_cols=["Hotel Name"],
    geo=True,
    tiles="EsriImagery",
    frame_width=800,
    frame_height=600,
    size="Humidity",
    scaling_factor=0.5,  # changed `scale` to `scaling_factor`
    color="City",
    title="City Humidity Heatmap"  # added closing parenthesis
)


# Display the map plot
map_plot_hotel


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)